## Исследование надёжности заёмщиков


### Шаг 1. Загрузка и оценка исходных данных

In [1]:
import pandas as pd
stat_payment = pd.read_csv('/datasets/data.csv')
print(stat_payment.head(10))
print()
print(stat_payment.info())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод

Данные в исходном виде не подготовлены к анализу, необходимо произвести их проверку на наличие пропусков, удалить дубликаты, перевести данные в формат удобный для обработки. Основные пропуски находятся в двух столбцах days_employed(общий трудовой стаж) и total_income(ежемесячный доход). Столбец days_employed(общий трудовой стаж) имеет формат данных float64(число с плавающей запятой, что для дней не характерно). Также в этом столбце встречаются отрицательные значения, которые не могут 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
stat_payment['days_employed'] = stat_payment['days_employed'].fillna(0)
stat_payment['total_income'] = stat_payment['total_income'].fillna(12130)
stat_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Мы заменили в столбцах days_employed, total_income отсутствующие значения на 0, так как  пропущеные значения являются технически не заполненными из-за того, что у человека может не быть трудового стажа или же не быть ежемесячного дохода. 

### Замена типа данных

In [3]:
stat_payment['days_employed'] = stat_payment['days_employed'].fillna(0)
stat_payment['total_income'] = stat_payment['total_income'].fillna(0)
stat_payment['days_employed'] = stat_payment['days_employed'].astype('int')
stat_payment['days_employed'] = abs(stat_payment['days_employed'])
stat_payment['children'] = abs(stat_payment['children'])  #переводим в положительные значения количество детей(оператор по ошибке добавил -)
stat_payment.loc[stat_payment['children'] == 20, 'children'] = 2 #заменяем значение 20 в столбце children на 2(оператор по ошибке добавил 0)
stat_payment.info() 
stat_payment['total_income'] =  stat_payment['total_income'].round(2) #Округляем доход до двух знаков после запятой       
print()
print(stat_payment['days_employed'].head(10))
print()
print(stat_payment['total_income'].head(10))
print()
print(stat_payment['children'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB

0      8437
1      4024
2      5623
3      4124
4    340266
5       926
6      2879
7       152
8      6929
9      2188
Name: days_employed, dtype: int64

0    253875.64
1    112080.01
2    145885.95
3    267628.55
4    158616.08
5    255763.57
6    240525.97
7    135823.93
8     95856.83
9    144425.94
Name: total_income, dtyp

### Вывод

Выбор метода astype() был обоснован тем, что нам необходимо было заменить вещественные значение столбца days_employed  на целочисленные для нормализации данных. Так как в дальнейшем для нашего исследования данный столбец не нужен, его предобработку мы на этом моменте остановим. Также округлим столбец total_income до двух знаков после запятой, для получения более наглядных значений дохода. Также переведем количество детей в столбце children в положительное значение, уберем артефакт 20 - переведем, его в 2, так как оператор при введении количества детей по ошибке нажал 0. 

### Обработка дубликатов

In [4]:
stat_payment_education = stat_payment['education'] 
stat_payment_education.value_counts()
stat_payment_education = stat_payment_education.str.lower()
stat_payment['education'] = stat_payment_education
print(stat_payment['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Обработаем столбец days_employed

In [5]:
stat_payment['days_employed'] = stat_payment['days_employed'].drop_duplicates() #Удаляем дубликаты в столбце days_employed
stat_payment = stat_payment.dropna().reset_index(drop = True)
stat_payment['days_employed'].duplicated().sum() #проверяем что дубликатов не осталось

Обработаем столбец total_income

In [6]:
#Удаляем дубликаты в столбце total_income
stat_payment['total_income'] = stat_payment['total_income'].drop_duplicates() 
stat_payment = stat_payment.dropna().reset_index(drop = True)
#проверяем что дубликатов не осталось
stat_payment['total_income'].duplicated().sum() 

Проверяем общее количество строк оставшееся в датафрейме stat_payment после удаления дубликатов 

In [7]:
stat_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

В столбце education(образование) встречались дубликаты вида(среднее, высшее, СРЕДНЕЕ, Среднее, неоконченное высшее, ВЫСШЕЕ, Высшее, начальное, Неоконченное высшее, НЕОКОНЧЕННОЕ ВЫСШЕЕ, НАЧАЛЬНОЕ, Начальное, ученая степень, Ученая степень, УЧЕНАЯ СТЕПЕНЬ) Так как различный регистр одних и тех же значений язык python считает уникальными, было принято решение создать переменную stat_payment_education в которую сохранить значение столбца stat_payment['education'] для получения объекта Siries и сначала подсчитать уникальные значения для подтверждения своей гипотезы, затем преобразовать все значения в нижний регистр методом str.lower()/ После этого присвоим значение переменной stat_payment_education столбцу stat_payment['education'] и проверим удаление дубликатов с разным регистром.
В столбцах days_employed  и total_income в учебных целях(так как в реальной жизни количество трудодней и доход у разных людей могут совпадать) найдем дубликаты и удалим их. После чего оценим датафрейм, его размер уменьшился более чем в два раза. 


### Лемматизация

Получаем список уникальных значений столбца purpose и выводим ее на экран для того чтобы оценить обьем и провести лемматизацию.

In [8]:
print(stat_payment['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Проведем лемматизацию столбца  purpose с помощью библиотеки pymystem3 и функции, получим новые значения 

In [9]:
purpose_category = stat_payment['purpose'].unique()
from pymystem3 import Mystem
m = Mystem()
def purpose_new(unique):
    lemmas_row = m.lemmatize(unique)
    if  'свадьба' in lemmas_row:
        return 'свадьба'
    if 'жилье' in lemmas_row or 'недвижимость' in lemmas_row:
        return 'недвижимость'
    if 'автомобиль' in lemmas_row:
        return 'автомобиль'
    if 'образование' in lemmas_row:
        return 'образование'
stat_payment['purpose_new'] = stat_payment['purpose'].apply(purpose_new) #Применение функции к столбцу purpose и запись новых значений в столбец purpose_new 
stat_payment['purpose'] = stat_payment['purpose_new'] #Присвоение значений столбца purpose_new столбцу purpose
stat_payment.drop(['purpose_new'], axis='columns', inplace=True) #Удаление ненужного столбца purpose_new
stat_payment.info()
stat_payment['purpose'].head(15)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


0     недвижимость
1       автомобиль
2     недвижимость
3      образование
4          свадьба
5     недвижимость
6     недвижимость
7      образование
8          свадьба
9     недвижимость
10    недвижимость
11    недвижимость
12         свадьба
13      автомобиль
14    недвижимость
Name: purpose, dtype: object

### Вывод

После проведения лемматизации столбца purpose(цель кредита) удалось оставить 4 основные цели на которые люди берут кредиты, а именно - свадьба, недвижимость, автомобиль, образование. Лематизация проводилась с помощью функции, которая проверяла наличие ключевых слов в исходном значении строки. После этого возвращала заданное значение в новый столбец. В последствии значения нового столбца были присвоены столбцу purpose. А столбец purpose_new был удален за ненадобностью  

### Категоризация данных

#### Влияние наличия детей на возврат кредита в срок

In [10]:
stat_payment_children = stat_payment[['children', 'debt','purpose']]

debtor_with_children = stat_payment_children.loc[
    (stat_payment_children['children'] != 0)&(
        stat_payment_children['debt'] == 1)]['debt'].count()

debtor_without_children = stat_payment_children.loc[
    (stat_payment_children['children'] == 0)&(
        stat_payment_children['debt'] == 1)]['debt'].count()

all_debtor = debtor_with_children + debtor_without_children
all_borrower = len(stat_payment_children)
all_debtor_percent = all_debtor / all_borrower

debtor_with_children_percent = debtor_with_children / all_borrower
debtor_without_children_percent = debtor_without_children / all_borrower
debtor_with_child_perc_all_debt = debtor_with_children  / all_debtor
debtor_without_child_perc_all_debt = debtor_without_children / all_debtor

#Ниже добавил вывод результатов в таблице, там можно оценить влияние количества детей на возврат кредита в срок
stat_payment_children_pivot = stat_payment_children.pivot_table(index = 'children', columns = 'debt', values = 'purpose', aggfunc = 'count')
stat_payment_children_pivot[1] = stat_payment_children_pivot[1].fillna(0)
stat_payment_children_pivot[0] = stat_payment_children_pivot[0].astype(int)
stat_payment_children_pivot[1] = stat_payment_children_pivot[1].astype(int)
stat_payment_children_pivot['share_of_debtors'] = stat_payment_children_pivot[1]/stat_payment_children_pivot[1].sum()

def compute_percentage(x):
      pct = float(x/stat_payment_children_pivot['share_of_debtors'].sum()) * 100
      return round(pct, 2)


stat_payment_children_pivot['percent_all_debt'] = stat_payment_children_pivot['share_of_debtors'].apply(compute_percentage)



print('Общее количество заемщиков', all_borrower)
print('Общее количество должников',all_debtor)
print('Количество должников с детьми:',debtor_with_children)
print('Количество должников без детей:',debtor_without_children) 
print('Общий процент должников от количества заемщиков: {:.2%}'.format(all_debtor_percent))
print('Процент должников с детьми от общего количества заемщиков: {:.2%}'.format(debtor_with_children_percent))
print('Процент должников без детей от общего количества заемщиков: {:.2%}'.format(debtor_without_children_percent))
print('Процент должников с детьми от общего количества должников: {:.2%}'.format(debtor_with_child_perc_all_debt))
print('Процент должников без детей от общего количества должников: {:.2%}'.format(debtor_without_child_perc_all_debt))
print()
print('Вывод в табличном виде результатов исследования')
stat_payment_children_pivot

Общее количество заемщиков 21525
Общее количество должников 1741
Количество должников с детьми: 678
Количество должников без детей: 1063
Общий процент должников от количества заемщиков: 8.09%
Процент должников с детьми от общего количества заемщиков: 3.15%
Процент должников без детей от общего количества заемщиков: 4.94%
Процент должников с детьми от общего количества должников: 38.94%
Процент должников без детей от общего количества должников: 61.06%

Вывод в табличном виде результатов исследования


debt,0,1,share_of_debtors,percent_all_debt
children,,,,
0,13086,1063,0.610569,61.06
1,4420,445,0.255600,25.56
2,1929,202,0.116025,11.60
3,303,27,0.015508,1.55
4,37,4,0.002298,0.23
5,9,0,0.000000,0.00


#### Влияние семейного положения на возврат кредита в срок(дополнительно исследовано влияние пола заемщика на возврат кредита в срок

In [19]:
stat_payment_family = stat_payment[['family_status', 'family_status_id', 'gender', 'debt']]

payment_married = stat_payment_family.loc[(stat_payment_family['family_status_id'] != 2)&(stat_payment_family['family_status_id'] != 3)&(stat_payment_family['family_status_id'] != 4)]
payment_single = stat_payment_family.loc[(stat_payment_family['family_status_id'] != 0)&(stat_payment_family['family_status_id'] != 1)]
debtor_married = payment_married.loc[payment_married['debt'] == 1]['debt'].count()
debtor_single = payment_single.loc[payment_single['debt'] == 1]['debt'].count()

debtor_married_all_percent = debtor_married / all_borrower
debtor_single_all_percent = debtor_single / all_borrower
debtor_married_debt_percent = debtor_married / all_debtor
debtor_single_debt_percent = debtor_single / all_debtor


gender_payment_women = stat_payment_family.loc[stat_payment_family['gender'] == 'F']
gender_payment_men = stat_payment_family.loc[stat_payment_family['gender'] == 'M']
gender_debt_women = gender_payment_women.loc[gender_payment_women['debt'] == 1]['debt'].count()
gender_debt_men = gender_payment_men.loc[gender_payment_men['debt'] == 1]['debt'].count()
gender_debt_women_all_perc = gender_debt_women / all_borrower
gender_debt_men_all_perc = gender_debt_men / all_borrower
gender_debt_women_debt_perc = gender_debt_women / all_debtor
gender_debt_men_debt_perc = gender_debt_men / all_debtor

#Добавление табличного вывода влияния семейного положения
stat_payment_family_pivot = stat_payment_family.pivot_table(index = 'family_status', columns = 'debt', values = 'gender', aggfunc = 'count')
stat_payment_family_pivot['share_of_debtors'] = stat_payment_family_pivot[1]/stat_payment_family_pivot[1].sum()

def compute_percentage(x):
      pct = float(x/stat_payment_family_pivot['share_of_debtors'].sum()) * 100
      return round(pct, 2)


stat_payment_family_pivot['percent_all_debt'] = stat_payment_family_pivot['share_of_debtors'].apply(compute_percentage)

#Добавление табличного вывода исследования влияния пола заемщика
stat_payment_gender_pivot = stat_payment_family.pivot_table(index = 'gender', columns = 'debt', values = 'family_status', aggfunc = 'count')
stat_payment_gender_pivot['share_of_debtors'] = stat_payment_gender_pivot[1]/stat_payment_gender_pivot[1].sum()

def compute_percentage(x):
      pct = float(x/stat_payment_gender_pivot['share_of_debtors'].sum()) * 100
      return round(pct, 2)


stat_payment_gender_pivot['percent_all_debt'] = stat_payment_gender_pivot['share_of_debtors'].apply(compute_percentage)


print('Общее количество заемщиков', all_borrower)
print('Общее количество должников',all_debtor)
print('Количество семейных заемщиков:',len(payment_married))
print('Количество несемейных заемщиков:',len(payment_single))
print('Количество семейных должников:',debtor_married)
print('Количество несемейных должников:',debtor_single)
print('Процент семейных должников от общего количества заемщиков: {:.2%}'.format(debtor_married_all_percent))
print('Процент несемейных должников от общего количества заемщиков: {:.2%}'.format(debtor_single_all_percent))
print('Процент семейных должников от общего количества должников: {:.2%}'.format(debtor_married_debt_percent ))
print('Процент несемейных должников от общего количества должников: {:.2%}'.format(debtor_single_debt_percent))
print('Количество заемщиков женщин:',len(gender_payment_women))
print('Количество заемщиков мужчин:',len(gender_payment_men))
print('Количество должников женщин:',gender_debt_women)
print('Количество должников мужчин:',gender_debt_men)
print('Процент должников женщин от общего количества заемщиков: {:.2%}'.format(gender_debt_women_all_perc))
print('Процент должников мужчин от общего количества заемщиков: {:.2%}'.format(gender_debt_men_all_perc))
print('Процент должников женщин от общего количества должников: {:.2%}'.format(gender_debt_women_debt_perc))
print('Процент должников мужчин от общего количества должников: {:.2%}'.format(gender_debt_men_debt_perc))
print()
print('Вывод в табличном виде результатов исследования влияния семейного положения')
print(stat_payment_family_pivot)
print()
print('Вывод в табличном виде результатов исследования влияния пола заемщика')
stat_payment_gender_pivot

Общее количество заемщиков 21525
Общее количество должников 1741
Количество семейных заемщиков: 16557
Количество несемейных заемщиков: 4968
Количество семейных должников: 1319
Количество несемейных должников: 422
Процент семейных должников от общего количества заемщиков: 6.13%
Процент несемейных должников от общего количества заемщиков: 1.96%
Процент семейных должников от общего количества должников: 75.76%
Процент несемейных должников от общего количества должников: 24.24%
Количество заемщиков женщин: 14236
Количество заемщиков мужчин: 7288
Количество должников женщин: 994
Количество должников мужчин: 747
Процент должников женщин от общего количества заемщиков: 4.62%
Процент должников мужчин от общего количества заемщиков: 3.47%
Процент должников женщин от общего количества должников: 57.09%
Процент должников мужчин от общего количества должников: 42.91%

Вывод в табличном виде результатов исследования влияния семейного положения
debt                       0    1  share_of_debtors  pe

debt,0,1,share_of_debtors,percent_all_debt
gender,,,,
F,13242.0,994.0,0.570936,57.09
M,6541.0,747.0,0.429064,42.91
XNA,1.0,NaN,NaN,NaN


<span style="color:green">
В исследовании про детей добавил табличный выод результатов, получилось, что 21 процент заемщиков имеющих просрочку, имеет 1 ребенка, а почти 70 процентов заемщиков вовсе не имеют детей не имеют.  

По семейному положению и полу, каких-то новых параметров в результате табличного вывода не добавилось
    
</span>

#### Влияние уровня дохода  на возврат кредита в срок

In [12]:
stat_payment_income = stat_payment[['total_income','debt']].copy() 
#Добавил метод .copy() по совету однокрусника, ругань пандас ушла.
stat_payment_income['income_level'] = ''
border_income = stat_payment_income['total_income'].quantile([.25, 0.5, .75]) 

#Алина, допилил категоризацию. Определяем границы для категорий дохода с помощью квантили,
#действительно крутая функция упрощающая работу

def income_group(income): 
    if income <= border_income[0.25]:
        return 'низкий доход'
    if  border_income[0.25]  <  income < border_income[0.5]:
        return 'средний доход'
    if  border_income[0.5]  <  income < border_income[0.75]:
        return 'высокий доход'
    return 'сверхвысокий доход'

#Применяем в качесте условия функции Series полученные методом квантили. Разбил людей по доходу на 4 категории. 


stat_payment_income['income_level'] = stat_payment_income['total_income'].apply(income_group)
stat_payment_income_pivot = stat_payment_income.pivot_table(index = 'income_level', columns = 'debt', values = 'total_income', aggfunc = 'count')
stat_payment_income_pivot['share_of_debtors'] = (stat_payment_income_pivot[1]/stat_payment_income_pivot[1].sum()) # Место где я изменил код

'''
Алина, вот тут я немного изменил функцию, раньше она считала процент должников как 
отношение должников к общему количеству заемщиков с определенным доходом, 
а надо считать отношение должников с конкретным доходом к общему количеству должников. 

Теперь то мы видим правильную процентовку в таблице    
'''

def compute_percentage(x):
      pct = float(x/stat_payment_income_pivot['share_of_debtors'].sum()) * 100
      return round(pct, 2)


stat_payment_income_pivot['percent_all_debt'] = stat_payment_income_pivot['share_of_debtors'].apply(compute_percentage)
stat_payment_income_pivot.head() 
#И, о да - она вертится!!! Но если быть честным, процентовка не сильно изменилась,
#как и раньше уровень дохода не влияет на возврат кредита в срок

debt,0,1,share_of_debtors,percent_all_debt
income_level,,,,
высокий доход,4894,486,0.279150,27.91
низкий доход,4965,417,0.239518,23.95
сверхвысокий доход,4997,386,0.221712,22.17
средний доход,4928,452,0.259621,25.96


#### Влияние целей кредитования  на возврат кредита в срок

In [13]:
stat_payment_target = stat_payment[['purpose','debt', 'total_income' ]]
stat_payment_target_pivot = stat_payment_target.pivot_table(index = 'purpose', columns = 'debt', values = 'total_income', aggfunc = 'count')
stat_payment_target_pivot['share_of_debtors'] = (stat_payment_target_pivot[1]/stat_payment_target_pivot[1].sum())

def compute_percentage(x): #Функция переводит в проценты от числа должников столбец доли должников в десятичном виде
      pct = float(x/stat_payment_target_pivot['share_of_debtors'].sum()) * 100
      return round(pct, 2)

stat_payment_target_pivot['percent_all_debt'] = stat_payment_target_pivot['share_of_debtors'].apply(compute_percentage)
stat_payment_target_pivot.head(10)

debt,0,1,share_of_debtors,percent_all_debt
purpose,,,,
автомобиль,3912,403,0.231476,23.15
недвижимость,10058,782,0.449167,44.92
образование,3652,370,0.212522,21.25
свадьба,2162,186,0.106835,10.68


### Вывод

Категоризация была проведена различными методами это метод выделения значений из датафрейма .loc для определения влияния на возврат кредита в срок наличия детей и семейного положения и применения функции в случае с влиянием уровня дохода, в задании с влиянием цели кредита использовалась сводная таблица  с использованием .pivot_table.

В задаче с наличием детей использовался словарь stat_payment[['children', 'debt']] - включающий столбец с количеством детей и столбцом наличия задолженности, для примененея к ним последовательно условий .loc. 

В задаче с семейным положением использовался словарь stat_payment[['family_status', 'family_status_id', 'gender', 'debt']] включающий: столбец с семейным положением, столбец идентификатор семейного положения, столбец пола(для доп ислледования), столбец наличия просроченных задолженностей для примененея к ним последовательно условий .loc. 

В задаче с влиянием уровня дохода использовался словарь stat_payment_income = stat_payment[['total_income','debt']], включающий столбец с доходом заемщиков для использования значений столбца в качестве аргумента функции, а также столбец наличия просроченных задолженностей, для вывода в сводной таблице.

В последней задаче с влиянием цели кредита на просрочку использовался словарь stat_payment_target = stat_payment[['purpose','debt', 'total_income' ]] включающий столбцы: Цель кредита, наличие просрочки, доход заемщика(использовался в качестве аргумента метода .pivot_table для подсчета значений.Столбцы использовались для вывода в сводной таблице 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
print('Общее количество заемщиков', all_borrower)
print('Общее количество должников',all_debtor)
print('Количество должников с детьми:',debtor_with_children)
print('Количество должников без детей:',debtor_without_children) 
print('Общий процент должников от количества заемщиков: {:.2%}'.format(all_debtor_percent))
print('Процент должников с детьми от общего количества заемщиков: {:.2%}'.format(debtor_with_children_percent))
print('Процент должников без детей от общего количества заемщиков: {:.2%}'.format(debtor_without_children_percent))
print('Процент должников с детьми от общего количества должников: {:.2%}'.format(debtor_with_child_perc_all_debt))
print('Процент должников без детей от общего количества должников: {:.2%}'.format(debtor_without_child_perc_all_debt))
# Алина, ниже доработанный код вывода
print()
print('Вывод в табличном виде результатов исследования')
print(stat_payment_children_pivot)

Общее количество заемщиков 21525
Общее количество должников 1741
Количество должников с детьми: 678
Количество должников без детей: 1063
Общий процент должников от количества заемщиков: 8.09%
Процент должников с детьми от общего количества заемщиков: 3.15%
Процент должников без детей от общего количества заемщиков: 4.94%
Процент должников с детьми от общего количества должников: 38.94%
Процент должников без детей от общего количества должников: 61.06%

Вывод в табличном виде результатов исследования
debt          0     1  share_of_debtors  percent_all_debt
children                                                 
0         13086  1063          0.610569             61.06
1          4420   445          0.255600             25.56
2          1929   202          0.116025             11.60
3           303    27          0.015508              1.55
4            37     4          0.002298              0.23
5             9     0          0.000000              0.00


### Вывод

Наличие детей у заемщиков не оказывает существенного влияния на возврат кредита в срок, а вот отсутствие детей наоборот, так как 2/3 должников - люди не имеющие детей 
<span style="color:green">
В результате доработки и табличного вывода результатов дополнительно выяснилось, что 21 процент заемщиков с просрочкой, люди имеющие 1 ребенка. Таким образом, люди у которых более 1 ребенка более ответственно относятся к возврату кредитов в срок. 
</span>

<span style="color:red">

Алина, 70 процентов людей с просрочками - люди без детей. А 21 процент имеет лишь одного ребенка. 
Получается люди с 2 и более детей, как раз ответственные, а вот без детей более легкомысленно относятся к возврату кредита
    
</span>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
print('Общее количество заемщиков', all_borrower)
print('Общее количество должников',all_debtor)
print('Количество семейных заемщиков:',len(payment_married))
print('Количество несемейных заемщиков:',len(payment_single))
print('Количество семейных должников:',debtor_married)
print('Количество несемейных должников:',debtor_single)
print('Процент семейных должников от общего количества заемщиков: {:.2%}'.format(debtor_married_all_percent))
print('Процент несемейных должников от общего количества заемщиков: {:.2%}'.format(debtor_single_all_percent))
print('Процент семейных должников от общего количества должников: {:.2%}'.format(debtor_married_debt_percent ))
print('Процент несемейных должников от общего количества должников: {:.2%}'.format(debtor_single_debt_percent))
print('Количество заемщиков женщин:',len(gender_payment_women))
print('Количество заемщиков мужчин:',len(gender_payment_men))
print('Количество должников женщин:',gender_debt_women)
print('Количество должников мужчин:',gender_debt_men)
print('Процент должников женщин от общего количества заемщиков: {:.2%}'.format(gender_debt_women_all_perc))
print('Процент должников мужчин от общего количества заемщиков: {:.2%}'.format(gender_debt_men_all_perc))
print('Процент должников женщин от общего количества должников: {:.2%}'.format(gender_debt_women_debt_perc))
print('Процент должников мужчин от общего количества должников: {:.2%}'.format(gender_debt_men_debt_perc))
# Добавил табличный вывод результатов возврата кредита в срок в зависимости от семейного положения и пола
print()
print('Вывод в табличном виде результатов исследования влияния пола заемщика')
stat_payment_gender_pivot

Общее количество заемщиков 21525
Общее количество должников 1741
Количество семейных заемщиков: 16557
Количество несемейных заемщиков: 4968
Количество семейных должников: 1319
Количество несемейных должников: 422
Процент семейных должников от общего количества заемщиков: 6.13%
Процент несемейных должников от общего количества заемщиков: 1.96%
Процент семейных должников от общего количества должников: 75.76%
Процент несемейных должников от общего количества должников: 24.24%
Количество заемщиков женщин: 14236
Количество заемщиков мужчин: 7288
Количество должников женщин: 994
Количество должников мужчин: 747
Процент должников женщин от общего количества заемщиков: 4.62%
Процент должников мужчин от общего количества заемщиков: 3.47%
Процент должников женщин от общего количества должников: 57.09%
Процент должников мужчин от общего количества должников: 42.91%

Вывод в табличном виде результатов исследования влияния пола заемщика


debt,0,1,share_of_debtors,percent_all_debt
gender,,,,
F,13242.0,994.0,0.570936,57.09
M,6541.0,747.0,0.429064,42.91
XNA,1.0,NaN,NaN,NaN


### Вывод

Наличие семьи негативно сказывается на возврате кредита в срок, почти 80 процентов людей имеющ их просрочку платежей - семейные люди. Также, стоит добавить, что в результате дополнительного исследования выявлено, что женщины составляют 60% от общего количества должников. То есть пол тоже оказывает влияние на возврат кредита в срок. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
stat_payment_income_pivot.head(10)

debt,0,1,share_of_debtors,percent_all_debt
income_level,,,,
высокий доход,4894,486,0.279150,27.91
низкий доход,4965,417,0.239518,23.95
сверхвысокий доход,4997,386,0.221712,22.17
средний доход,4928,452,0.259621,25.96


### Вывод

Уровень дохода не оказывает существенного влияния на возврат кредита в срок, как видно из таблицы, процентное соотношение людей имеющих разный уровень дохода и имеющих просрочку примерно одинаково

- Как разные цели кредита влияют на его возврат в срок?

In [17]:
stat_payment_target_pivot.head(10)

debt,0,1,share_of_debtors,percent_all_debt
purpose,,,,
автомобиль,3912,403,0.231476,23.15
недвижимость,10058,782,0.449167,44.92
образование,3652,370,0.212522,21.25
свадьба,2162,186,0.106835,10.68


### Вывод

Цели кредита тоже влияют на возврат кредита в срок в примерно равной степени, единственное, должников немного меньше в категории "недвижимость". 
<span style="color:red">
Ввиду исправления подсчета процентов по категориям от общего количества должников, оказалось что цель кредита влияет на возврат кредита в срок. Так 45 процентов людей имеющих просрочки берут кредит на недвижимость.  
</span>

 


### Шаг 4. Общий вывод

В результате проведенного иссследования, выявлено что: 
1. Наличие детей не влияет на возврат кредита в срок, а их отсутствие наоборот (70% должников не имеют детей)<span style="color:green">21 процент имеет лишь одного ребенка. Получается люди с 2 и более детей, как раз ответственные, а вот без детей более легкомысленно относятся к возврату кредита</span> 

2. Семейное положение оказывает существенное влияние на возврат кредита в срок (почти 80% должников - женатые/замужние люди или люди проживающие в гражданском браке, также выяснилось, что 60 процентов должников - женского пола)

3. Размер дохода не оказывает существенного влияния на возврат кредита в срок.(Доли людей имеющих просрочку распределены равномерно 

4. Цели кредитования влияют на возврат кредита примерно в равной степени.<span style="color:red">Ввиду исправления расчета  процентов от общего количества должников - оказалось, что цель кредита влияет на его возврат в срок, так 45 процентов людей имеющих просрочку по кредиту взяли его на недвижимость.</span> 

Таким образом, факторы влияющие на возврат кредита в срок: семейное положение и отсутствие детей. Согласно выводам исследования в группу риска по кредитам попадают: 

**Замужние(проживающие в гражданском браке) женщины не имеющие детей <span style="color:green">или имеющие 1 ребенка, которые взяли кредит на недвижимость</span>**
